In [25]:
import os
import cv2
import shutil
import random
from pathlib import Path
from ultralytics import YOLO


In [29]:
IMG_PATH = Path(r'./img')
train_val_test = (0.8,0.1,0.1)
ANNOTATION_PATH = Path(r'./annotation')
YOLO_ANNOTATION = Path(r'./yolo_annotation')

class_dict = {category_name: idx for idx, category_name in enumerate(os.listdir(IMG_PATH))}
print(class_dict)

{'People': 0, 'Motorbike': 1, 'Cup': 2, 'Bicycle': 3, 'Chair': 4, 'Boat': 5, 'Table': 6, 'Car': 7, 'Bottle': 8, 'Bus': 9, 'Cat': 10, 'Dog': 11}


In [35]:
def corresponding_image_path(folder_path, annotaion_path):
    file_title, _ = os.path.splitext(annotaion_path)
    return IMG_PATH / folder_path / file_title

def pascal_to_yolo():
    for sub_category in os.listdir(ANNOTATION_PATH):
        folder_path = ANNOTATION_PATH / sub_category
        for file_name in os.listdir(folder_path):
            file_path = folder_path / file_name
            
            yolo_annotation_path = YOLO_ANNOTATION/sub_category/file_name
            if os.path.isfile(yolo_annotation_path):
                continue

            os.makedirs(YOLO_ANNOTATION/sub_category, exist_ok=True)

            image_path = corresponding_image_path(sub_category, file_name)
            img = cv2.imread(image_path)
            if img is None:
                print(f"{file_path} cannot be read.")
                continue

            img_width, img_height = img.shape[:2]

            with open(file_path, 'r') as anno_file:
                pascal_data = anno_file.readlines()[1:]

            pascal_num = list(map(lambda s: s.strip().split(" ")[:5], pascal_data))
            
            yolo_format = []
            for indv_bounding_coordinates in pascal_num:
                indv_bounding_coordinates[0] = class_dict[indv_bounding_coordinates[0]]
                indv_bounding_coordinates[1:] = list(map(int, indv_bounding_coordinates[1:]))

                
                class_category = indv_bounding_coordinates[0]
                x_ctr = (indv_bounding_coordinates[1] + indv_bounding_coordinates[3]/2) / img_width
                y_ctr = (indv_bounding_coordinates[2] + indv_bounding_coordinates[4]/2) / img_height
                width = (indv_bounding_coordinates[3]) / img_width
                height = (indv_bounding_coordinates[4]) / img_height

                yolo_format.append([class_category, x_ctr, y_ctr, width, height]) 

            with open(yolo_annotation_path, 'w') as yolo_writer:
                for data in yolo_format:
                    yolo_writer.write(f"{' '.join(list(map(str, data)))}\n")
                
            

pascal_to_yolo()

libpng warning: iCCP: known incorrect sRGB profile


annotation/Bicycle/2015_00391.jpg.txt cannot be read.


[ WARN:0@3837.731] global loadsave.cpp:268 findDecoder imread_('img/Bicycle/2015_00391.jpg'): can't open/read file: check file path/integrity
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: cHRM chunk does not match sRGB
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: cHRM chunk does not match sRGB
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


In [ ]:

image_files = list((dataset_path / "images").glob("*.jpg"))  # or .png
random.shuffle(image_files)
split_idx = int(len(image_files) * split_ratio)
train_images = image_files[:split_idx]
val_images = image_files[split_idx:]

def copy_split(images, split_name):
    for img_path in images:
        label_path = dataset_path / "labels" / (img_path.stem + ".txt")
        if label_path.exists():
            shutil.copy(img_path, yolo_data_path / split_name / "images" / img_path.name)
            shutil.copy(label_path, yolo_data_path / split_name / "labels" / label_path.name)

copy_split(train_images, "train")
copy_split(val_images, "val")


In [ ]:

# === STEP 3: CREATE YOLO CONFIG FILE ===
data_yaml = yolo_data_path / "data.yaml"
data_yaml.write_text(f"""\
path: {yolo_data_path}
train: train/images
val: val/images
names: {classes}
""")


In [ ]:

# === STEP 4: TRAIN YOLOv8 MODEL ===
# You can use yolov8n.yaml, yolov8s.yaml, yolov8m.yaml, etc.
model = YOLO("yolov8n.pt")  # Using pretrained YOLOv8n weights

model.train(
    data=str(data_yaml),
    epochs=50,
    imgsz=640,
    batch=16,
    name="custom_yolov8n_run"
)
